# Import datasets taken from MOH

In [1]:
import pandas as pd

In [2]:
cases_df = pd.read_csv("cases_state.csv")
cases_df.head()

date            state  cases_new  cases_import  cases_recovered  \
0  2020-01-25            Johor          4             4                0   
1  2020-01-25            Kedah          0             0                0   
2  2020-01-25         Kelantan          0             0                0   
3  2020-01-25           Melaka          0             0                0   
4  2020-01-25  Negeri Sembilan          0             0                0   

   cases_active  cases_cluster  cases_unvax  cases_pvax  cases_fvax  ...  \
0             4              0            4           0           0  ...   
1             0              0            0           0           0  ...   
2             0              0            0           0           0  ...   
3             0              0            0           0           0  ...   
4             0              0            0           0           0  ...   

   cases_0_4  cases_5_11  cases_12_17  cases_18_29  cases_30_39  cases_40_49  \
0          0           0            0            0            0            1   
1          0           0            0            0            0            0   
2          0           0            0            0            0            0   
3          0           0            0            0            0            0   
4          0           0            0            0            0            0   

   cases_50_59  cases_60_69  cases_70_79  cases_80  
0            0            0            0         0  
1            0            0            0         0  
2            0            0            0         0  
3            0            0            0         0  
4            0            0            0         0  

[5 rows x 25 columns]

In [3]:
deaths_df = pd.read_csv("deaths_state.csv")
deaths_df.head()

date            state  deaths_new  deaths_bid  deaths_new_dod  \
0  2020-03-17            Johor           1           0               1   
1  2020-03-17            Kedah           0           0               0   
2  2020-03-17         Kelantan           0           0               0   
3  2020-03-17           Melaka           0           0               0   
4  2020-03-17  Negeri Sembilan           0           0               0   

   deaths_bid_dod  deaths_unvax  deaths_pvax  deaths_fvax  deaths_boost  \
0               0             1            0            0             0   
1               0             0            0            0             0   
2               0             0            0            0             0   
3               0             0            0            0             0   
4               0             0            0            0             0   

   deaths_tat  
0           0  
1           0  
2           0  
3           0  
4           0

In [4]:
beds_df = pd.read_csv("hospital.csv")
beds_df.head()

date            state  beds  beds_covid  beds_noncrit  admitted_pui  \
0  2020-03-24            Johor  1440         434          1408             0   
1  2020-03-24            Kedah  1218         143          1183             0   
2  2020-03-24         Kelantan  1463         280          1424             9   
3  2020-03-24           Melaka  1091          82          1065             0   
4  2020-03-24  Negeri Sembilan  1223         253          1205             0   

   admitted_covid  admitted_total  discharged_pui  discharged_covid  \
0               0               0               0                 0   
1               0               0               0                 0   
2               6              15               1                 0   
3               0               0               0                 0   
4               0               0               0                 0   

   discharged_total  hosp_covid  hosp_pui  hosp_noncovid  
0                 0          18         9              1  
1                 0          46         5             12  
2                 1          78        58             23  
3                 0          19        10             12  
4                 0          90        13              8

In [5]:
icu_df = pd.read_csv("icu.csv")
icu_df.head()

date            state  beds_icu  beds_icu_rep  beds_icu_total  \
0  2020-03-24            Johor        10            22              32   
1  2020-03-24            Kedah        35             0              35   
2  2020-03-24         Kelantan        39             0              39   
3  2020-03-24           Melaka        26             0              26   
4  2020-03-24  Negeri Sembilan        18             0              18   

   beds_icu_covid  vent  vent_port  icu_covid  icu_pui  icu_noncovid  \
0              10    11          2          0        0             1   
1              16    34          0          2        1            12   
2              20    42          4          1        0            20   
3              12    23          4          0        2            12   
4              10    14          9          0        0             8   

   vent_covid  vent_pui  vent_noncovid  vent_used  vent_port_used  
0           0         0              0          8               0  
1           1         1              0         19               0  
2           1         0              0         26               0  
3           0         1              0         13               5  
4           0         0              0          1               6

Drop extra columns and convert date to datetime

In [6]:
cases_df = cases_df.loc[:, cases_df.columns.intersection(['date','state', 'cases_new'])]
cases_df['date'] = pd.to_datetime(cases_df['date'])
cases_df.head()

date            state  cases_new
0 2020-01-25            Johor          4
1 2020-01-25            Kedah          0
2 2020-01-25         Kelantan          0
3 2020-01-25           Melaka          0
4 2020-01-25  Negeri Sembilan          0

In [7]:
deaths_df = deaths_df.loc[:, deaths_df.columns.intersection(['date','state', 'deaths_new'])]
deaths_df['date'] = pd.to_datetime(deaths_df['date'])
deaths_df.head()

date            state  deaths_new
0 2020-03-17            Johor           1
1 2020-03-17            Kedah           0
2 2020-03-17         Kelantan           0
3 2020-03-17           Melaka           0
4 2020-03-17  Negeri Sembilan           0

In [8]:
beds_df = pd.merge(beds_df, icu_df, on=['date','state'])
beds_df = beds_df.loc[:, beds_df.columns.intersection(['date','state', 'beds', 'beds_covid', 'beds_icu_covid'])]
beds_df['date'] = pd.to_datetime(beds_df['date'])
beds_df.head()

date            state  beds  beds_covid  beds_icu_covid
0 2020-03-24            Johor  1440         434              10
1 2020-03-24            Kedah  1218         143              16
2 2020-03-24         Kelantan  1463         280              20
3 2020-03-24           Melaka  1091          82              12
4 2020-03-24  Negeri Sembilan  1223         253              10

# Combine datasets

In [9]:
from datetime import date, timedelta, datetime
# format: y/m/d
sdate = date(2020,1,1)
edate = date(2022,2,21)

states = list(cases_df['state'].unique())

new_df=pd.DataFrame(columns=['date','state', 'cases_new', 'deaths_new', 'beds', 'beds_covid', 'beds_icu_covid'])
row_count = 0
for date in pd.date_range(sdate,edate-timedelta(days=1),freq='d'):
    for state in states:
        row_count += 1
        new_df.loc[row_count, 'date'] = datetime.strftime(date, '%Y-%m-%d')
        new_df.loc[row_count, 'state'] = state
# new_df['date'] = pd.date_range(sdate,edate-timedelta(days=1),freq='d')
new_df.head(20)

date              state cases_new deaths_new beds beds_covid  \
1   2020-01-01              Johor       NaN        NaN  NaN        NaN   
2   2020-01-01              Kedah       NaN        NaN  NaN        NaN   
3   2020-01-01           Kelantan       NaN        NaN  NaN        NaN   
4   2020-01-01             Melaka       NaN        NaN  NaN        NaN   
5   2020-01-01    Negeri Sembilan       NaN        NaN  NaN        NaN   
6   2020-01-01             Pahang       NaN        NaN  NaN        NaN   
7   2020-01-01              Perak       NaN        NaN  NaN        NaN   
8   2020-01-01             Perlis       NaN        NaN  NaN        NaN   
9   2020-01-01       Pulau Pinang       NaN        NaN  NaN        NaN   
10  2020-01-01              Sabah       NaN        NaN  NaN        NaN   
11  2020-01-01            Sarawak       NaN        NaN  NaN        NaN   
12  2020-01-01           Selangor       NaN        NaN  NaN        NaN   
13  2020-01-01         Terengganu       NaN        NaN  NaN        NaN   
14  2020-01-01  W.P. Kuala Lumpur       NaN        NaN  NaN        NaN   
15  2020-01-01        W.P. Labuan       NaN        NaN  NaN        NaN   
16  2020-01-01     W.P. Putrajaya       NaN        NaN  NaN        NaN   
17  2020-01-02              Johor       NaN        NaN  NaN        NaN   
18  2020-01-02              Kedah       NaN        NaN  NaN        NaN   
19  2020-01-02           Kelantan       NaN        NaN  NaN        NaN   
20  2020-01-02             Melaka       NaN        NaN  NaN        NaN   

   beds_icu_covid  
1             NaN  
2             NaN  
3             NaN  
4             NaN  
5             NaN  
6             NaN  
7             NaN  
8             NaN  
9             NaN  
10            NaN  
11            NaN  
12            NaN  
13            NaN  
14            NaN  
15            NaN  
16            NaN  
17            NaN  
18            NaN  
19            NaN  
20            NaN

Take data from old datasets into new datasets.

In [10]:
import datetime
def transpose(new_df, old_df, cols):
    for index, row in new_df.iterrows():
        current_date = new_df.loc[index, 'date']
        current_state = new_df.loc[index, 'state']
        for index1, row1 in old_df.loc[old_df['date'] == current_date].iterrows():
            state = old_df.loc[index1, 'state']
            if current_state == state:
                for col in cols:
                    new_df.loc[index, col] = old_df.loc[index1, col]

transpose(new_df, cases_df, ['cases_new'])
new_df.tail(20)

date              state cases_new deaths_new beds beds_covid  \
12493  2022-02-19         Terengganu       616        NaN  NaN        NaN   
12494  2022-02-19  W.P. Kuala Lumpur      1247        NaN  NaN        NaN   
12495  2022-02-19        W.P. Labuan       368        NaN  NaN        NaN   
12496  2022-02-19     W.P. Putrajaya       144        NaN  NaN        NaN   
12497  2022-02-20              Johor      2644        NaN  NaN        NaN   
12498  2022-02-20              Kedah      2254        NaN  NaN        NaN   
12499  2022-02-20           Kelantan      1605        NaN  NaN        NaN   
12500  2022-02-20             Melaka       713        NaN  NaN        NaN   
12501  2022-02-20    Negeri Sembilan       690        NaN  NaN        NaN   
12502  2022-02-20             Pahang      1581        NaN  NaN        NaN   
12503  2022-02-20              Perak       358        NaN  NaN        NaN   
12504  2022-02-20             Perlis       113        NaN  NaN        NaN   
12505  2022-02-20       Pulau Pinang      1811        NaN  NaN        NaN   
12506  2022-02-20              Sabah      5007        NaN  NaN        NaN   
12507  2022-02-20            Sarawak       309        NaN  NaN        NaN   
12508  2022-02-20           Selangor      7180        NaN  NaN        NaN   
12509  2022-02-20         Terengganu       608        NaN  NaN        NaN   
12510  2022-02-20  W.P. Kuala Lumpur      1428        NaN  NaN        NaN   
12511  2022-02-20        W.P. Labuan       358        NaN  NaN        NaN   
12512  2022-02-20     W.P. Putrajaya       173        NaN  NaN        NaN   

      beds_icu_covid  
12493            NaN  
12494            NaN  
12495            NaN  
12496            NaN  
12497            NaN  
12498            NaN  
12499            NaN  
12500            NaN  
12501            NaN  
12502            NaN  
12503            NaN  
12504            NaN  
12505            NaN  
12506            NaN  
12507            NaN  
12508            NaN  
12509            NaN  
12510            NaN  
12511            NaN  
12512            NaN

In [11]:
transpose(new_df, deaths_df, ['deaths_new'])
new_df.tail(20)

date              state cases_new deaths_new beds beds_covid  \
12493  2022-02-19         Terengganu       616          1  NaN        NaN   
12494  2022-02-19  W.P. Kuala Lumpur      1247          0  NaN        NaN   
12495  2022-02-19        W.P. Labuan       368          0  NaN        NaN   
12496  2022-02-19     W.P. Putrajaya       144          0  NaN        NaN   
12497  2022-02-20              Johor      2644          9  NaN        NaN   
12498  2022-02-20              Kedah      2254          6  NaN        NaN   
12499  2022-02-20           Kelantan      1605          0  NaN        NaN   
12500  2022-02-20             Melaka       713          1  NaN        NaN   
12501  2022-02-20    Negeri Sembilan       690          0  NaN        NaN   
12502  2022-02-20             Pahang      1581          2  NaN        NaN   
12503  2022-02-20              Perak       358          4  NaN        NaN   
12504  2022-02-20             Perlis       113          0  NaN        NaN   
12505  2022-02-20       Pulau Pinang      1811          3  NaN        NaN   
12506  2022-02-20              Sabah      5007          4  NaN        NaN   
12507  2022-02-20            Sarawak       309          0  NaN        NaN   
12508  2022-02-20           Selangor      7180          7  NaN        NaN   
12509  2022-02-20         Terengganu       608          0  NaN        NaN   
12510  2022-02-20  W.P. Kuala Lumpur      1428          0  NaN        NaN   
12511  2022-02-20        W.P. Labuan       358          1  NaN        NaN   
12512  2022-02-20     W.P. Putrajaya       173          0  NaN        NaN   

      beds_icu_covid  
12493            NaN  
12494            NaN  
12495            NaN  
12496            NaN  
12497            NaN  
12498            NaN  
12499            NaN  
12500            NaN  
12501            NaN  
12502            NaN  
12503            NaN  
12504            NaN  
12505            NaN  
12506            NaN  
12507            NaN  
12508            NaN  
12509            NaN  
12510            NaN  
12511            NaN  
12512            NaN

In [12]:
transpose(new_df, beds_df, ['beds', 'beds_covid', 'beds_icu_covid'])
new_df.tail(20)

date              state cases_new deaths_new  beds beds_covid  \
12493  2022-02-19         Terengganu       616          1  1593        401   
12494  2022-02-19  W.P. Kuala Lumpur      1247          0  3963        582   
12495  2022-02-19        W.P. Labuan       368          0   202         98   
12496  2022-02-19     W.P. Putrajaya       144          0   609         73   
12497  2022-02-20              Johor      2644          9  5209       1473   
12498  2022-02-20              Kedah      2254          6  2664       1259   
12499  2022-02-20           Kelantan      1605          0  2684        410   
12500  2022-02-20             Melaka       713          1  1315        651   
12501  2022-02-20    Negeri Sembilan       690          0  1749       1084   
12502  2022-02-20             Pahang      1581          2  1992        655   
12503  2022-02-20              Perak       358          4  5152        652   
12504  2022-02-20             Perlis       113          0   503        111   
12505  2022-02-20       Pulau Pinang      1811          3  2057        597   
12506  2022-02-20              Sabah      5007          4  4342       1717   
12507  2022-02-20            Sarawak       309          0  3571        678   
12508  2022-02-20           Selangor      7180          7  5378       1203   
12509  2022-02-20         Terengganu       608          0  1593        421   
12510  2022-02-20  W.P. Kuala Lumpur      1428          0  3963        582   
12511  2022-02-20        W.P. Labuan       358          1   202         98   
12512  2022-02-20     W.P. Putrajaya       173          0   609         73   

      beds_icu_covid  
12493             40  
12494             22  
12495              7  
12496              6  
12497             46  
12498            130  
12499             36  
12500             24  
12501             61  
12502             81  
12503             55  
12504             11  
12505             42  
12506            122  
12507             71  
12508             74  
12509             40  
12510             26  
12511              7  
12512              6

In [13]:
new_df.dtypes

date              object
state             object
cases_new         object
deaths_new        object
beds              object
beds_covid        object
beds_icu_covid    object
dtype: object

In [14]:
new_df[["cases_new", "deaths_new", "beds", "beds_covid", "beds_icu_covid"]] = new_df[["cases_new", "deaths_new", "beds", "beds_covid", "beds_icu_covid"]].apply(pd.to_numeric)
new_df.dtypes

date               object
state              object
cases_new         float64
deaths_new        float64
beds              float64
beds_covid        float64
beds_icu_covid    float64
dtype: object

In [15]:
new_df=new_df.sort_values(['date']).reset_index(drop=True)
new_df["cum_cases"]=new_df.groupby(['state'])['cases_new'].cumsum(axis=0)
new_df["cum_deaths"]=new_df.groupby(['state'])['deaths_new'].cumsum(axis=0)
new_df.sort_values(['date', 'state'])
new_df.tail(20)

date              state  cases_new  deaths_new    beds  \
12492  2022-02-19           Kelantan     1885.0         1.0  2684.0   
12493  2022-02-19              Kedah     2429.0         2.0  2664.0   
12494  2022-02-19              Johor     2715.0         8.0  5209.0   
12495  2022-02-19             Perlis      191.0         0.0   503.0   
12496  2022-02-20  W.P. Kuala Lumpur     1428.0         0.0  3963.0   
12497  2022-02-20         Terengganu      608.0         0.0  1593.0   
12498  2022-02-20           Selangor     7180.0         7.0  5378.0   
12499  2022-02-20            Sarawak      309.0         0.0  3571.0   
12500  2022-02-20              Sabah     5007.0         4.0  4342.0   
12501  2022-02-20       Pulau Pinang     1811.0         3.0  2057.0   
12502  2022-02-20             Perlis      113.0         0.0   503.0   
12503  2022-02-20    Negeri Sembilan      690.0         0.0  1749.0   
12504  2022-02-20             Pahang     1581.0         2.0  1992.0   
12505  2022-02-20             Melaka      713.0         1.0  1315.0   
12506  2022-02-20           Kelantan     1605.0         0.0  2684.0   
12507  2022-02-20              Kedah     2254.0         6.0  2664.0   
12508  2022-02-20              Johor     2644.0         9.0  5209.0   
12509  2022-02-20        W.P. Labuan      358.0         1.0   202.0   
12510  2022-02-20              Perak      358.0         4.0  5152.0   
12511  2022-02-20     W.P. Putrajaya      173.0         0.0   609.0   

       beds_covid  beds_icu_covid  cum_cases  cum_deaths  
12492       410.0            36.0   199711.0      1293.0  
12493      1259.0           130.0   209684.0      2213.0  
12494      1473.0            46.0   298900.0      3977.0  
12495       111.0            11.0     9900.0       139.0  
12496       582.0            26.0   239491.0      2706.0  
12497       421.0            40.0    91686.0       769.0  
12498      1203.0            74.0   896324.0     10118.0  
12499       678.0            71.0   256377.0      1623.0  
12500      1717.0           122.0   308476.0      2904.0  
12501       597.0            42.0   192580.0      1787.0  
12502       111.0            11.0    10013.0       139.0  
12503      1084.0            61.0   134082.0      1349.0  
12504       655.0            81.0   118117.0       819.0  
12505       651.0            24.0    92219.0       983.0  
12506       410.0            36.0   201316.0      1293.0  
12507      1259.0           130.0   211938.0      2219.0  
12508      1473.0            46.0   301544.0      3986.0  
12509        98.0             7.0    14078.0       152.0  
12510       652.0            55.0   140735.0      1478.0  
12511        73.0             6.0    12704.0        22.0

In [16]:
new_df['state'] = new_df['state'].replace(['Pulau Pinang'],'Penang')
new_df['state'] = new_df['state'].replace(['W.P. Kuala Lumpur'],'Kuala Lumpur')
new_df['state'] = new_df['state'].replace(['W.P. Labuan'],'Labuan')
new_df['state'] = new_df['state'].replace(['W.P. Putrajaya'],'Putrajaya')
new_df['state'] = new_df['state'].replace(['Melaka'],'Malacca')
new_df.tail(20)

date            state  cases_new  deaths_new    beds  beds_covid  \
12492  2022-02-19         Kelantan     1885.0         1.0  2684.0       410.0   
12493  2022-02-19            Kedah     2429.0         2.0  2664.0      1259.0   
12494  2022-02-19            Johor     2715.0         8.0  5209.0      1473.0   
12495  2022-02-19           Perlis      191.0         0.0   503.0       111.0   
12496  2022-02-20     Kuala Lumpur     1428.0         0.0  3963.0       582.0   
12497  2022-02-20       Terengganu      608.0         0.0  1593.0       421.0   
12498  2022-02-20         Selangor     7180.0         7.0  5378.0      1203.0   
12499  2022-02-20          Sarawak      309.0         0.0  3571.0       678.0   
12500  2022-02-20            Sabah     5007.0         4.0  4342.0      1717.0   
12501  2022-02-20           Penang     1811.0         3.0  2057.0       597.0   
12502  2022-02-20           Perlis      113.0         0.0   503.0       111.0   
12503  2022-02-20  Negeri Sembilan      690.0         0.0  1749.0      1084.0   
12504  2022-02-20           Pahang     1581.0         2.0  1992.0       655.0   
12505  2022-02-20          Malacca      713.0         1.0  1315.0       651.0   
12506  2022-02-20         Kelantan     1605.0         0.0  2684.0       410.0   
12507  2022-02-20            Kedah     2254.0         6.0  2664.0      1259.0   
12508  2022-02-20            Johor     2644.0         9.0  5209.0      1473.0   
12509  2022-02-20           Labuan      358.0         1.0   202.0        98.0   
12510  2022-02-20            Perak      358.0         4.0  5152.0       652.0   
12511  2022-02-20        Putrajaya      173.0         0.0   609.0        73.0   

       beds_icu_covid  cum_cases  cum_deaths  
12492            36.0   199711.0      1293.0  
12493           130.0   209684.0      2213.0  
12494            46.0   298900.0      3977.0  
12495            11.0     9900.0       139.0  
12496            26.0   239491.0      2706.0  
12497            40.0    91686.0       769.0  
12498            74.0   896324.0     10118.0  
12499            71.0   256377.0      1623.0  
12500           122.0   308476.0      2904.0  
12501            42.0   192580.0      1787.0  
12502            11.0    10013.0       139.0  
12503            61.0   134082.0      1349.0  
12504            81.0   118117.0       819.0  
12505            24.0    92219.0       983.0  
12506            36.0   201316.0      1293.0  
12507           130.0   211938.0      2219.0  
12508            46.0   301544.0      3986.0  
12509             7.0    14078.0       152.0  
12510            55.0   140735.0      1478.0  
12511             6.0    12704.0        22.0

In [17]:
new_df.to_csv("daily.csv", index=False)